In [145]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import unicodedata

In [110]:
url = "http://www.francetvinfo.fr/elections/resultats/"
resultats = requests.get(url)
soup = BeautifulSoup(resultats.text, 'html.parser')
df = pd.DataFrame(columns=["depart", "code", "circo", "nom", "parti", "score"])
circos_url = []

index=0


for depart in soup.find("section", {"id": "departmentResults"}).findAll("a", href=True):
    url_depart = depart["href"]
    print(url_depart)
    
    soup_depart = BeautifulSoup(requests.get(url_depart).text, 'html.parser')
    depart_name = url_depart.split("/")[-2][:-3]
    depart_number = url_depart.split("_")[-1][:-1]
    i=0
    for circo in (soup_depart.findAll("section")[3].findAll("a", href=True)):
        circo_number = i
        url_circo = circo["href"]
        
        if url_circo in circos_url:
            continue
        else : 
            circos_url.append(url_circo)
            i+=1
        
        try:
            soup_circo = BeautifulSoup(requests.get(url_circo).text, 'html.parser')
        except:
            print("error at : ", url_circo)
            continue
            
        candidats = soup_circo.findAll("section")[3].findAll("div", {"class" : re.compile("^candidate")})[:-2]
        
        for candidat in candidats: 
            try:
                name = candidat.find("span", {"class": "name"}).text
                parti = candidat.find("span", {"class": "party"}).text
                score = candidat.find("span", {"class": "percent"}).text
                df.loc[index] = [depart_name, depart_number, circo_number, name, parti, score]
                index+=1
            except :
                print("error at ", depart_name, circo_number)
                continue



http://www.francetvinfo.fr/elections/resultats/ain_01/
http://www.francetvinfo.fr/elections/resultats/aisne_02/
http://www.francetvinfo.fr/elections/resultats/allier_03/
http://www.francetvinfo.fr/elections/resultats/alpes-de-haute-provence_04/
http://www.francetvinfo.fr/elections/resultats/hautes-alpes_05/
http://www.francetvinfo.fr/elections/resultats/alpes-maritimes_06/
http://www.francetvinfo.fr/elections/resultats/ardeche_07/
http://www.francetvinfo.fr/elections/resultats/ardennes_08/
http://www.francetvinfo.fr/elections/resultats/ariege_09/
http://www.francetvinfo.fr/elections/resultats/aube_10/
http://www.francetvinfo.fr/elections/resultats/aude_11/
http://www.francetvinfo.fr/elections/resultats/aveyron_12/
http://www.francetvinfo.fr/elections/resultats/bouches-du-rhone_13/
http://www.francetvinfo.fr/elections/resultats/calvados_14/
http://www.francetvinfo.fr/elections/resultats/cantal_15/
http://www.francetvinfo.fr/elections/resultats/charente_16/
http://www.francetvinfo.fr/ele

In [111]:
df.code.unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
       '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
       '88', '89', '90', '91', '92', '93', '94', '95', '971', '972', '975',
       '976', '977', '986', '987', '988'], dtype=object)

In [112]:
df.shape

(7560, 6)

In [181]:
df2 = df.copy()

In [182]:
url_alamano = ["http://www.francetvinfo.fr/elections/resultats/la-reunion_974/la-reunion_1ere-circonscription",
              "http://www.francetvinfo.fr/elections/resultats/la-reunion_974/la-reunion_2eme-circonscription", 
              "http://www.francetvinfo.fr/elections/resultats/la-reunion_974/la-reunion_3eme-circonscription",
              "http://www.francetvinfo.fr/elections/resultats/la-reunion_974/la-reunion_4eme-circonscription",
              "http://www.francetvinfo.fr/elections/resultats/la-reunion_974/la-reunion_5eme-circonscription",
              "http://www.francetvinfo.fr/elections/resultats/la-reunion_974/la-reunion_6eme-circonscription",
              "http://www.francetvinfo.fr/elections/resultats/la-reunion_974/la-reunion_7eme-circonscription",
              "http://www.francetvinfo.fr/elections/resultats/guyane_973/guyane_1ere-circonscription",
              "http://www.francetvinfo.fr/elections/resultats/guyane_973/guyane_2eme-circonscription"]
for i, url in enumerate(url_alamano[-2:]):
    soup_circo = BeautifulSoup(requests.get(url).text, 'html.parser')
    for item in soup_circo.findAll("section")[3].findAll("div", {"class" : re.compile("^candidate")})[:-2]:
        name = item.find("span", {"class": "name"}).text
        parti = item.find("span", {"class": "party"}).text
        score = item.find("span", {"class": "percent"}).text
        df2.loc[df2.shape[0]+1] = ["guyane", 973, i, name, parti, score]


In [183]:
df2.shape

(7579, 6)

In [184]:
df2.head()

,depart,code,circo,nom,parti,score
0,ain,01,0.0,Laurent Mallet,Modem,"33,89 %"
1,ain,01,0.0,Xavier Breton,Les Républicains,"26,78 %"
2,ain,01,0.0,Jérôme Buisson,Front National,"15,46 %"
3,ain,01,0.0,Fabrine Martin Zemlik,La France insoumise,"9,70 %"
4,ain,01,0.0,Florence Blatrix-Contat,Parti socialiste,"9,23 %"


In [185]:
df2.score = df2.score.map(lambda x : float(x.replace(",", ".").replace("%", "").strip()))

In [186]:
df2.nom = df2.nom.map(lambda s : unicodedata.normalize('NFD', s).encode('ascii', 'ignore').decode("utf-8") )

In [187]:
df2["prenom"] = df2.nom.map(lambda x : x.split(" ")[0])
df2.nom = df2.nom.map(lambda x : x.split(" ")[1])

In [189]:
df2.circo +=1

In [190]:
df2.head()

,depart,code,circo,nom,parti,score,prenom
0,ain,01,1.0,Mallet,Modem,33.89,Laurent
1,ain,01,1.0,Breton,Les Républicains,26.78,Xavier
2,ain,01,1.0,Buisson,Front National,15.46,Jerome
3,ain,01,1.0,Martin,La France insoumise,9.70,Fabrine
4,ain,01,1.0,Blatrix-Contat,Parti socialiste,9.23,Florence


In [192]:
df2.to_csv("data/resultats_1ertour.csv", index=False)